In [10]:
!pip install qiskit qiskit_aer --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 95.0 MB/s eta 0:00:00


In [30]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

num_bits = 15

alice_bits = np.random.randint(2, size=num_bits)

alice_bases = np.random.randint(2, size=num_bits)

print("Alice's Bits:  ", alice_bits)
print("Alice's Bases: ", alice_bases)

def encode_qubits(bits, bases):
    qubits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)  # Apply X gate if bit is 1
        if basis == 1:
            qc.h(0)  # Apply Hadamard if basis is 1 (X-basis)
        qubits.append(qc)
    return qubits

# Alice prepares the qubits
alice_qubits = encode_qubits(alice_bits, alice_bases)

# Bob chooses random bases
bob_bases = np.random.randint(2, size=num_bits)

def measure_qubits(qubits, bases):
    measured_bits = []
    sim = AerSimulator()  # Quantum simulator

    for qc, basis in zip(qubits, bases):
        qc_copy = qc.copy()  # Copy circuit to avoid modifying original

        if basis == 1:
            qc_copy.h(0)  # Apply Hadamard if measuring in X basis

        qc_copy.measure(0, 0)  # Measure qubit

        # Transpile and run on simulator
        transpiled_qc = transpile(qc_copy, sim)
        job = sim.run(transpiled_qc, shots=1)
        result = job.result()

        measured_bit = int(list(result.get_counts().keys())[0])  # Extract measurement result
        measured_bits.append(measured_bit)

    return np.array(measured_bits)

bob_bits = measure_qubits(alice_qubits, bob_bases)

print("Bob's Bases:  ", bob_bases)
print("Bob's Bits:   ", bob_bits)
matching_indices = np.where(alice_bases == bob_bases)[0]

alice_key = alice_bits[matching_indices]
bob_key = bob_bits[matching_indices]

print("\n🔹 Matching Indices: ", matching_indices)
print("🔑 Alice's Key: ", alice_key)
print("🔑 Bob's Key:   ", bob_key)

sample_size = max(1, len(alice_key) // 4)  # Check 25% of the key
sample_indices = np.random.choice(len(alice_key), sample_size, replace=False)

alice_sample = alice_key[sample_indices]
bob_sample = bob_key[sample_indices]

print("\n🔍 Sample Check (Publicly Compared Bits):")
print("Alice's Sample:", alice_sample)
print("Bob's Sample:  ", bob_sample)

if not np.array_equal(alice_sample, bob_sample):
    print("\n⚠️ Possible Eavesdropping Detected! Discard Key.")
else:
    final_key = np.delete(alice_key, sample_indices)
    print("\n✅ Final Shared Secret Key: ", final_key)


Alice's Bits:   [1 1 1 0 1 0 1 0 1 1 1 0 1 1 0]
Alice's Bases:  [0 0 0 1 0 0 0 1 1 1 1 0 0 1 1]
Bob's Bases:   [1 1 0 0 0 1 1 1 1 0 1 0 1 0 0]
Bob's Bits:    [1 0 1 1 1 0 1 0 1 1 1 0 1 0 0]

🔹 Matching Indices:  [ 2  4  7  8 10 11]
🔑 Alice's Key:  [1 1 0 1 1 0]
🔑 Bob's Key:    [1 1 0 1 1 0]

🔍 Sample Check (Publicly Compared Bits):
Alice's Sample: [1]
Bob's Sample:   [1]

✅ Final Shared Secret Key:  [1 1 0 1 0]



🔹 Matching Indices:  [ 1  3  9 11 12 14]
🔑 Alice's Key:  [1 1 0 1 0 0]
🔑 Bob's Key:    [1 1 0 1 0 0]

🔍 Sample Check (Publicly Compared Bits):
Alice's Sample: [0]
Bob's Sample:   [0]

✅ Final Shared Secret Key:  [1 1 0 1 0]
